# Cleaning Data

In the following code I will be cleaning the data provided within two excel spreadsheets that were provided by the City of Missoula. These spreadsheets contain budget information for the City of Missoula and aligining Program Inventory information surrounding different programs that are funded by the City of Missoula. 

The data provided is messy and comes from both the financial software "Tyler Edens" and Workiva WDesk. The data in the excel files is messy and unusable to perform proper analysis and understanding. This process will load the files into the environment, remove unneccesary information, perform data cleaning functions and ultimately prepare the data for visualization in Power Bi. 

In Power BI, I will create a dashboard that provides information about all of the different programs currently funded and provide granular breakdowns for financial and other important information therein. 

Once the data is cleaned I will conduct some exploratory analysis on the fresh data to see both how the cleaning process performed but also to showcase lurking information within the data that is easily digestable at a glance. 

## Step 1: Importing The Libraries
This code will bring forth all the libraries needed for data wrangling, cleaning, and visualization. 

In [ ]:
# Standard Libraries
import os

# Text Cleaning
import re

# Data Handling
import pandas as pd
import numpy as np

# Excel Handling
import openpyxl

# Additional Cleaning Utilities; each labeled below
from tqdm import tqdm  # Adds progress bars to loops
import chardet  # Detects encoding issues
import janitor # Chaining and Cleaing Helpers

# Visualization
import missingno as msno
import matplotlib.pyplot as plt
import matplotlib.ticker as mtick

### Step 2: Loading The Files
The code below will load all the necessary Excel files that will be cleaned in this project.

In [7]:
# define file path
data_path = "data/"

# Load Files; only the first sheet in expenditure and program inventory, all of the sheets in revenue expense. 
revenue_expense = pd.read_excel(
    os.path.join(data_path, "FY24_Revenue_Expense_Data.xlsx"), 
    sheet_name=None, engine="openpyxl")

expenditure_status = pd.read_excel(
    os.path.join(data_path, "FY24_Expenditure_Status.xlsx"),
    sheet_name=0,engine="openpyxl")

program_inventory = pd.read_excel(
    os.path.join(data_path, "Program_Inventory_Internal_Data_Collection.xlsx"), 
    sheet_name=0, engine="openpyxl")

In [ ]:
# Preview Expenditure Status
expenditure_status

,Unnamed: 0,Unnamed: 1,Unnamed: 2,Unnamed: 3,Unnamed: 4,Unnamed: 5,Unnamed: 6,Unnamed: 7,Unnamed: 8,Unnamed: 9,Unnamed: 10,Unnamed: 11,Unnamed: 12,Unnamed: 13,Unnamed: 14,Unnamed: 15,Unnamed: 16
0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,,,,,,,,
4,1000,GENERAL FUND,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
14133,Total DEPRECIATION,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
14134,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
14135,Total GENERAL FIXED ASSETS ACCOUNT GROUP,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
14136,Grand Total,338487828.71,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


### Step 3: Clean Raw Expenditure Status File (Initial Filtering Logic)
This next step takes the above file and filters out header artifacts, summary rows, and other non-transactional records using logical rules. 

In [ ]:
# Condition 1: Keep rows where column A "Unnamed: 0" has at least one digit
condition_1 = expenditure_status["Unnamed: 0"].astype(str).str.contains(r"\d", na=False)

# Condition 2: Column B "Unnamed: 1" must not be empty
condition_2 = expenditure_status["Unnamed: 1"].notna()

# Condition 3: Remove rows where column A is numeric and column B is just text
condition_3 = ~(
    expenditure_status["Unnamed: 0"].astype(str).str.isnumeric() &
    expenditure_status["Unnamed: 1"].astype(str).str.isalpha()
)

# Condition 4: Remove rows where both column A and column B are only alphabetic
condition_4 = ~(
    expenditure_status["Unnamed: 0"].astype(str).str.isalpha() &
    expenditure_status["Unnamed: 1"].astype(str).str.isalpha()
)

# Condition 5: Keep only rows where column B contains a number (ensures a dollar amount)
condition_5 = expenditure_status["Unnamed: 1"].astype(str).str.contains(r"\d", na=False)

# Condition 6: Exclude rows where column A contains the word "Total"
condition_6 = ~expenditure_status["Unnamed: 0"].astype(str).str.contains("Total", case=False, na=False)

# Combine all logic conditions
all_conditions = condition_1 & condition_2 & condition_3 & condition_4 & condition_5 & condition_6

# Subset the filtered data
expenditure_filtered_df = expenditure_status[all_conditions].copy()

# Only keep relevant first two columns
expenditure_filtered_df = expenditure_filtered_df.iloc[:, :2]

# Rename columns for clarity
expenditure_filtered_df.columns = ["account_number", "adjusted_appropriation"]

# Preview the cleaned result
expenditure_filtered_df.head()

,account_number,adjusted_appropriation
9,1000.210.410100.110.000 SALARIES AND WAGES,214440
10,1000.210.410100.140.000 EMPLOYER CONTRIBUTIONS,199915
11,1000.210.410100.141.000 STATE RETIREMENT CON...,217
15,1000.210.410100.210.000 OFFICE SUPPLIES,200
16,1000.210.410100.230.000 REPAIR/MAINTENANCE,750


In [ ]:
## Display Preview of Files after Loading and pre-cleaning
### Print sheet names in revenue/expense Excel file
print("Revenue Expense Loaded:", revenue_expense.keys())

### Preview filtered expenditure data (after smart logic filters)
print("\n Filtered Expenditure Status Preview:")
print(expenditure_filtered_df.head())

### Preview raw program inventory data (survey structure)
print("\n Program Inventory Preview:")
print(program_inventory.head())

In [ ]:
## Define Cleaning Functions
def drop_unnamed_columns(df):
    """
    Drops columns with headers like 'Unnamed: XX'.
    These are usually trailing blank columns from Excel exports.
    """
    return df.loc[:, ~df.columns.str.contains("^Unnamed", na=False)]


def clean_numeric_column(column, zfill=None):
    """
    Remove trailing '.0' from numeric-like strings.
    Optionally left-pad with zeros if a zfill length is provided.
    """
    cleaned = column.astype(str).str.replace(r"\.0$", "", regex=True)
    return cleaned.str.zfill(zfill) if zfill else cleaned


def clean_identifiers(df):
    """
    Standardize key identifiers like 'Fund #', 'Dept #' and 'Activity Code'.
    Applies formatting and zero-padding where appropriate.
    """
    if "Fund #" in df.columns:
        df["Fund #"] = clean_numeric_column(df["Fund #"])
    if "Dept #" in df.columns:
        df["Dept #"] = clean_numeric_column(df["Dept #"], zfill=3)
    if "Activity Code" in df.columns:
        df["Activity Code"] = clean_numeric_column(df["Activity Code"], zfill=6)
    return df


def expand_multicolumn_headers(df, column_expansion_map):
    """
    Expands wide compound headers into multiple real columns.
    For example, "Mandate (E41, H41, E43)" → [mandate_e41, mandate_h41, mandate_e43].
    Assumes the next N columns are 'Unnamed'.
    """
    new_columns = []
    col_iter = iter(df.columns)

    while True:
        try:
            col = next(col_iter)
            if col in column_expansion_map:
                expanded = column_expansion_map[col]
                new_columns.extend(expanded)
                for _ in range(len(expanded) - 1):
                    next(col_iter)
            else:
                new_columns.append(col)
        except StopIteration:
            break

    df.columns = new_columns
    return df


def apply_department_and_fund_mappings(df, dept_map, fund_map):
    """
    Applies mapping dictionaries to generate human-readable department/fund names.
    Falls back to 'REDACTED' if no match is found.
    """
    df["Department"] = df["Dept #"].map(dept_map).fillna("REDACTED")
    if "Fund" in df.columns:
        df["Fund Name"] = df["Fund"].map(lambda f: fund_map.get(f, "REDACTED"))
    return df


def clean_program_inventory(df, column_expansion_map, dept_map, fund_map):
    """
    Full cleaning pipeline for the Program Inventory dataset.
    Renames key columns, applies identifier formatting,
    expands compound headers, drops filler columns,
    and applies department/fund mapping.
    """
    df = df.rename(columns={"Org": "Dept #"})
    df = clean_identifiers(df)
    df = expand_multicolumn_headers(df, column_expansion_map)
    df = drop_unnamed_columns(df)
    df = apply_department_and_fund_mappings(df, dept_map, fund_map)
    return df


def remove_leading_trailing_whitespace(df):
    """
    Trim whitespace from all string-like columns only.
    Safely handles object columns that are not strings.
    """
    for col in df.select_dtypes(include="object").columns:
        df[col] = df[col].astype(str).str.strip()
    return df

def standardize_case(df, cols):
    """
    Convert selected columns to title case.
    Useful for labels like 'department' and 'fund name'.
    """
    for col in cols:
        if col in df.columns:
            df[col] = df[col].astype(str).str.title()
    return df

def strip_whitespace_and_standardize(df):
    """
    Combo function to:
    - Remove leading/trailing whitespace from text
    - Title-case key label fields
    """
    df = remove_leading_trailing_whitespace(df)
    df = standardize_case(df, cols=["Department", "Department (H6)", "Fund Name"])
    return df

In [ ]:
## When using the comments in line, it allows you to easily reference them later by using help(function)
## help(fix_multiline_headers)

In [ ]:
## Step 7: Data Cleaning Process for Expenditure Status

# Start from the filtered, clean transactional rows
df_expenditure_status = expenditure_filtered_df.copy()

# Rename columns for clarity
df_expenditure_status.columns = ["Account Number", "Adjusted Appropriation"]

# Split the structured account number into separate identifiers
split_cols = ["Fund #", "Dept #", "Activity Code", "Object Code", "Sub-object Code"]
split_data = df_expenditure_status["Account Number"].astype(str).str.split(".", expand=True, n=4)
split_data.columns = split_cols

# Add split columns to main DataFrame
df_expenditure_status = pd.concat([df_expenditure_status, split_data], axis=1)

# Convert identifiers to nullable integer (preserves blanks as <NA>)
for col in split_cols:
    df_expenditure_status[col] = pd.to_numeric(df_expenditure_status[col], errors="coerce").astype("Int64")

# Forward-fill Fund and Dept across rows
df_expenditure_status[["Fund #", "Dept #"]] = df_expenditure_status[["Fund #", "Dept #"]].ffill()

# Drop rows missing Activity Code — indicates header artifacts
df_expenditure_status = df_expenditure_status.dropna(subset=["Activity Code"]).reset_index(drop=True)

# Extract the Account Description from Account Number text
# Assumes description comes after the last space
df_expenditure_status["Account Description"] = df_expenditure_status["Account Number"].astype(str).str.extract(r"(?:\d{3,}\.\d{3,}(?:\.\d{3,}){1,3}\s+)(.*)", expand=False)

# Strip leading/trailing spaces in the new column
df_expenditure_status["Account Description"] = df_expenditure_status["Account Description"].str.strip()

# Final preview of structured data
print("\n✅ Structured Expenditure Data Preview:")
display(df_expenditure_status.head(10))

In [ ]:
# Preview column names to see where compound headers are followed by Unnamed cols
print("\nRaw Program Inventory Columns:")
for i, col in enumerate(program_inventory.columns):
    print(f"{i}: {col}")

In [ ]:
## Step 8: Generate Mapping Dictionaries for Department & Fund Names

# Extract unique Dept and Fund codes from cleaned expenditure data
dept_mapping_df = df_expenditure_status[["Dept #"]].drop_duplicates().dropna()
fund_mapping_df = df_expenditure_status[["Fund #"]].drop_duplicates().dropna()

# Manually create department name lookup dictionary
dept_name_lookup = {
    210: "City Council", 220: "Mayor", 230: "Finance", 240: "Human Resources",
    250: "Legal", 260: "Police", 270: "Fire", 280: "Public Works",
    290: "Parks & Recreation", 300: "Planning & Development", 310: "Library",
    320: "IT Services", 330: "Community Development", 340: "Housing Services"
}

# Build final dictionaries from available codes
# These map numeric fund/department values to names
dept_map = dept_mapping_df["Dept #"].astype(int).map(dept_name_lookup).to_dict()
fund_map = fund_mapping_df["Fund #"].dropna().astype(str).to_dict()

# Apply human-readable department names to main DataFrame
df_expenditure_status["Department"] = df_expenditure_status["Dept #"].map(dept_name_lookup).fillna("REDACTED")


In [ ]:
## Step 9: Final Whitespace Cleanup and Standardization

# Strip whitespace and title case key label fields
df_expenditure_status = strip_whitespace_and_standardize(df_expenditure_status)

# Preview cleaned structure
print("\n✅ Final Cleaned Expenditure Status Sample:")
display(df_expenditure_status.head(5))

In [ ]:
## Step 10: Normalize All Column Headers with pyjanitor

# Normalize expenditure_status (already exists)
df_expenditure_status = df_expenditure_status.clean_names()

# Only clean program_inventory if it was already processed
if 'df_program_inventory' in globals():
    df_program_inventory = df_program_inventory.clean_names()

# Normalize each sheet in revenue_expense workbook
for sheet_name, df in revenue_expense.items():
    revenue_expense[sheet_name] = df.clean_names()

# Preview normalized columns
print("\n📌 Expenditure Status Columns:")
print(df_expenditure_status.columns.tolist())

if 'df_program_inventory' in globals():
    print("\n📌 Program Inventory Columns:")
    print(df_program_inventory.columns.tolist())
else:
    print("\n⚠️ Skipping preview for df_program_inventory (not yet defined)")

In [ ]:
## Step 11: Clean Program Inventory (Survey-Based Intake)

# Define Multi Header expansion mapping for program inventory
column_expansion_map = {
    "Cost Recovery (E58, P24)": ["cost_recovery_e58", "cost_recovery_p24"],
    "Mandate (E41, H41, E43)": ["mandate_e41", "mandate_h41", "mandate_e43"],
    "Service Level (E47, H47, E49)": ["service_level_e47", "service_level_h47", "service_level_e49"],
    "Reliance & Interdependencies (E53, E55)": ["reliance_e53", "reliance_e55"],
    "Strategic Goal (E64, E66, E68, E74, E80)": [
        "strategic_goal_e64", "strategic_goal_e66", "strategic_goal_e68",
        "strategic_goal_e74", "strategic_goal_e80"
    ],
    "Trend (Demand) (E87, E89)": ["trend_demand_e87", "trend_demand_e89"],
    "Risk (E93, E95)": ["risk_e93", "risk_e95"]
}

# Rename 'Org' to 'Dept #' for consistency with expenditure dataset
program_inventory = program_inventory.rename(columns={"Org": "Dept #"})

# Clean identifiers (fund, dept, activity)
program_inventory = clean_identifiers(program_inventory)

# Expand compound headers into multiple columns using our defined map
program_inventory = expand_multicolumn_headers(program_inventory, column_expansion_map)

# Drop unnecessary unnamed columns left over from Excel export
program_inventory = drop_unnamed_columns(program_inventory)

# Map readable department and fund names
df_program_inventory = apply_department_and_fund_mappings(
    program_inventory,
    dept_map=dept_map,
    fund_map=fund_map
)

# Final standardization for labels and whitespace
df_program_inventory = strip_whitespace_and_standardize(df_program_inventory)

# Preview final column structure
print("\n✅ Cleaned Program Inventory Columns:")
for i, col in enumerate(df_program_inventory.columns):
    print(f"{i}: {col}")

In [ ]:
## Normalize column headers using pyjanitor (snake_case, consistent labels)

df_expenditure_status = df_expenditure_status.clean_names()
df_program_inventory = df_program_inventory.clean_names()

# Normalize each sheet in the Revenue/Expense workbook
for sheet_name, df in revenue_expense.items():
    revenue_expense[sheet_name] = df.clean_names()

## Confirm cleaned headers for downstream use (Power BI or merges)

print("\n📌 Expenditure Status Columns:")
print(df_expenditure_status.columns.tolist())

print("\n📌 Program Inventory Columns:")
print(df_program_inventory.columns.tolist())


In [ ]:
## Add Validations

# Summary of nulls in key datasets
print("🔎 Nulls in Expenditure Status:")
print(df_expenditure_status.isnull().sum())

print("\n🔎 Nulls in Program Inventory:")
print(df_program_inventory.isnull().sum())

# Check data types (unexpected types might mean parsing issues)
print("\n📌 Data Types - Expenditure Status:")
print(df_expenditure_status.dtypes)

print("\n📌 Data Types - Program Inventory:")
print(df_program_inventory.dtypes)

In [ ]:
# Ensure all Fund #, Dept #, and Activity Codes are not null
missing_keys = df_expenditure_status[
    df_expenditure_status[["fund_#", "dept_#", "activity_code"]].isnull().any(axis=1)
]
print(f"\nMissing key budget identifiers: {missing_keys.shape[0]} rows")

# Spot duplicate account numbers (could mean bad export or repeats)
duplicates = df_expenditure_status[df_expenditure_status.duplicated(subset="account_number", keep=False)]
print(f"\nDuplicate account numbers: {duplicates.shape[0]}")

In [ ]:
# ✅ Export each cleaned dataset as its own CSV
df_expenditure_status.to_csv("cleaned_expenditure_status.csv", index=False)
df_program_inventory.to_csv("cleaned_program_inventory.csv", index=False)

print("✅ All cleaned files saved individually.")